Instale las bibliotecas necesarias para realizar el web scraping. La biblioteca más popular en Python para esto es requests para obtener páginas web y BeautifulSoup o Scrapy para analizar el HTML.

In [1]:
pip install requests

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/5c/f2/f3faa20684729d3910af2ee142e30432c7a46a817eadeeab87366ed87bbb/charset_normalizer-3.2.0-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 61.5/61.5 kB 3.2 MB/s eta 0:00:00
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/9b/81/62fd61001fa4b9d0df6e31d47ff49cfa9de4af03adecf339c7bc30656b37/urllib3-2.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d13177aaa050113286e93e7b40eae01fbf7c3d9/certifi-2023.7.22-py3-none-any.whl.metadata
   ---------------------------------------- 62.6/62.6 kB 3

Y a su vez BeautifulSoup4, esta biblioteca ayuda a analizar el HTML y extraer datos de manera más fácil y estructurada.

In [2]:
pip install beautifulsoup4

     -------------------------------------- 143.0/143.0 kB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


De la seccion Top 100 de la pagina extraigo los titulos y URL de cada libro, les coloco Id segun su posicion en el top

In [1]:
import requests
from bs4 import BeautifulSoup

# URL del top 100 de libros
url = 'https://www.cuspide.com/cienmasvendidos'

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa 
# (código 200 esto significa OK, es decir que la solicitud fue procesada correctamente por el servidor)
if response.status_code == 200:
    # Analizar el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrar todos los elementos <article>
    articulos = soup.find_all('article')

    # Extraer los datos de los libros de los elementos <article>
    libros = []
    id = 0
    for articulo in articulos:
        titulo = articulo.find('a')['title']
        url = articulo.find('a')['href']
        
        # Creo un diccionario que en cada ciclo se actualiza con los datos de los libros
        libro = {
            'id' : id,
            'titulo': titulo,
            'url' : 'https://www.cuspide.com/' + url
        }
        # Cada vez que se actualice el diccionario este se va a agregar a la lista, llenandose asi de todos los datos de los libros
        libros.append(libro)

        id += 1
        
    # Mostrar los datos de los libros
    for libro in libros:
        print(f"Id: {libro['id']}")
        print(f"Título: {libro['titulo']}")
        print(f"URL: {libro['url']}")

else:
    print('Error al acceder a la página:', response.status_code)


Id: 0
Título: Este Dolor No Es Mio
URL: https://www.cuspide.com//Libro/9788484456810/Este+Dolor+No+Es+Mio
Id: 1
Título: Una Familia Anormal
URL: https://www.cuspide.com//Libro/9789877364262/Una+Familia+Anormal
Id: 2
Título: Adios Cachorra
URL: https://www.cuspide.com//Libro/9789504979395/Adios+Cachorra
Id: 3
Título: El Viento Conoce Mi Nombre
URL: https://www.cuspide.com//Libro/9789500769013/El+Viento+Conoce+Mi+Nombre
Id: 4
Título: Born Y Quieto
URL: https://www.cuspide.com//Libro/9789504979456/Born+Y+Quieto
Id: 5
Título: Antes De Que Se Enfrie El Cafe
URL: https://www.cuspide.com//Libro/9789506446567/Antes+De+Que+Se+Enfrie+El+Cafe
Id: 6
Título: Habitos Atomicos
URL: https://www.cuspide.com//Libro/9789501298321/Habitos+Atomicos
Id: 7
Título: El Hombre En Busca De Sentido
URL: https://www.cuspide.com//Libro/9788425432026/El+Hombre+En+Busca+De+Sentido
Id: 8
Título: Encuentra Tu Persona Vitamina
URL: https://www.cuspide.com//Libro/9789508523334/Encuentra+Tu+Persona+Vitamina
Id: 9
Título: 

De cada libro en especifico extraigo su precio en pesos y dolares, como figuran en la pagina

In [2]:
#Funcion obtener_precios desfazada para comprobar el funcionamiento y lo que retorna ars_span
urlp = 'https://www.cuspide.com/Libro/9789877364262/Una+Familia+Anormal'
response = requests.get(urlp)
response.status_code == 200
soup = BeautifulSoup(response.text, 'html.parser')
precio_container = soup.find('div', class_='md-precio')
ars_span = precio_container.find('span', string='AR$').next_sibling.get_text(strip = True).replace('.', '').replace(',', '.')
print(ars_span)

4199.00


In [3]:
import requests
from bs4 import BeautifulSoup

def obtener_precios(urlp):
    response = requests.get(urlp)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        precio_container = soup.find('div', class_='md-precio') # Encuentra el div con la clase md-precio
        if precio_container:
            # En AR$ se necesitan dos replace para sacar los puntos como separador de miles y reemplazar las comas por puntos como separadores de decimales
            ars_span = precio_container.find('span', string='AR$').next_sibling.get_text(strip = True).replace('.', '').replace(',', '.') # Encuentra el span que contenga AR$ y selecciona su hermano siguiente, que serian los digitos del precio
            #En U$s solo con colocar el replace de comas por puntos como separadores de miles bastaria porque no hay libros que lleguen a la unidad de miles, pero si en algun momento llega a ese monto no va a tener que modificarse nada
            usd_span = precio_container.find('span', string='U$s').next_sibling.get_text(strip = True).replace('.', '').replace(',', '.') 
        return ars_span, usd_span
    return "Precio no encontrado", "Precio no encontrado"

# Obtener y mostrar precios
precios = []
for librop in libros: #Utilizo la lista de libros creada anteriormente para iterar los url y asi extraer el precio de cada uno de forma automática
    nombre = librop['titulo']
    urlp = librop['url']
    ars_precio, usd_precio = obtener_precios(urlp)

    # Creo un diccionario que en cada ciclo se actualiza con los precios de los libros
    precio = {
        'AR$' : ars_precio,
        'U$s' : usd_precio
    }
    # En cada ciclo los datos dentro del diccionario se van a agregar a la lista, llenandose asi de todos los precios de los libros
    precios.append(precio)
    
    print(f"Libro: {nombre}")
    print(f"Precio en AR$: {precio['AR$']}")
    print(f"Precio en U$s: {precio['U$s']}")


Libro: Este Dolor No Es Mio
Precio en AR$: 9900.00
Precio en U$s: 27.09
Libro: Una Familia Anormal
Precio en AR$: 4199.00
Precio en U$s: 11.49
Libro: Adios Cachorra
Precio en AR$: 6900.00
Precio en U$s: 18.88
Libro: El Viento Conoce Mi Nombre
Precio en AR$: 9999.00
Precio en U$s: 27.36
Libro: Born Y Quieto
Precio en AR$: 8900.00
Precio en U$s: 24.35
Libro: Antes De Que Se Enfrie El Cafe
Precio en AR$: 7999.00
Precio en U$s: 21.89
Libro: Habitos Atomicos
Precio en AR$: 10900.00
Precio en U$s: 29.82
Libro: El Hombre En Busca De Sentido
Precio en AR$: 4900.00
Precio en U$s: 13.41
Libro: Encuentra Tu Persona Vitamina
Precio en AR$: 8900.00
Precio en U$s: 24.35
Libro: Alas De Sangre
Precio en AR$: 8900.00
Precio en U$s: 24.35
Libro: La Argentina En El Fondo
Precio en AR$: 7450.00
Precio en U$s: 20.38
Libro: Nosotros Dos En La Tormenta
Precio en AR$: 8999.00
Precio en U$s: 24.62
Libro: Tengase Presente
Precio en AR$: 7900.00
Precio en U$s: 21.61
Libro: El Amor Es Imposible
Precio en AR$: 850

Combino la lista de libros con la de precios para que me quede una sola lista con esos datos juntos

In [4]:
libros_precios = []

for dicc in range(len(libros)):
    dicc_combinado = {**libros[dicc], **precios[dicc]}
    libros_precios.append(dicc_combinado)

print(libros_precios)


[{'id': 0, 'titulo': 'Este Dolor No Es Mio', 'url': 'https://www.cuspide.com//Libro/9788484456810/Este+Dolor+No+Es+Mio', 'AR$': '9900.00', 'U$s': '27.09'}, {'id': 1, 'titulo': 'Una Familia Anormal', 'url': 'https://www.cuspide.com//Libro/9789877364262/Una+Familia+Anormal', 'AR$': '4199.00', 'U$s': '11.49'}, {'id': 2, 'titulo': 'Adios Cachorra', 'url': 'https://www.cuspide.com//Libro/9789504979395/Adios+Cachorra', 'AR$': '6900.00', 'U$s': '18.88'}, {'id': 3, 'titulo': 'El Viento Conoce Mi Nombre', 'url': 'https://www.cuspide.com//Libro/9789500769013/El+Viento+Conoce+Mi+Nombre', 'AR$': '9999.00', 'U$s': '27.36'}, {'id': 4, 'titulo': 'Born Y Quieto', 'url': 'https://www.cuspide.com//Libro/9789504979456/Born+Y+Quieto', 'AR$': '8900.00', 'U$s': '24.35'}, {'id': 5, 'titulo': 'Antes De Que Se Enfrie El Cafe', 'url': 'https://www.cuspide.com//Libro/9789506446567/Antes+De+Que+Se+Enfrie+El+Cafe', 'AR$': '7999.00', 'U$s': '21.89'}, {'id': 6, 'titulo': 'Habitos Atomicos', 'url': 'https://www.cuspi

Separo los libros que tienen precios de los que no se pudo scrapear el precio

In [5]:
import datetime

errores = []
idError = 0
for i in libros_precios:
    nombre = i['titulo']
    urli = i['url']
    precio_pesos = i['AR$']
    precio_dolares = i['U$s']
    fecha = datetime.date.today()
    error = {
        'idError':idError,
        'titulo':i['titulo'],
        'url':i['url'],
        'fecha':fecha.strftime("%Y-%m-%d")
    }
    if (precio_pesos == 'Precio no encontrado') and (precio_dolares == 'Precio no encontrado'):
        errores.append(error)
        libro_a_eliminar = {'titulo': i['titulo'], 'url': i['url']}
        libros_precios.remove(libro_a_eliminar)
        idError += 1
    else:
        continue
print(errores)
print(libros_precios)
    

[]
[{'id': 0, 'titulo': 'Este Dolor No Es Mio', 'url': 'https://www.cuspide.com//Libro/9788484456810/Este+Dolor+No+Es+Mio', 'AR$': '9900.00', 'U$s': '27.09'}, {'id': 1, 'titulo': 'Una Familia Anormal', 'url': 'https://www.cuspide.com//Libro/9789877364262/Una+Familia+Anormal', 'AR$': '4199.00', 'U$s': '11.49'}, {'id': 2, 'titulo': 'Adios Cachorra', 'url': 'https://www.cuspide.com//Libro/9789504979395/Adios+Cachorra', 'AR$': '6900.00', 'U$s': '18.88'}, {'id': 3, 'titulo': 'El Viento Conoce Mi Nombre', 'url': 'https://www.cuspide.com//Libro/9789500769013/El+Viento+Conoce+Mi+Nombre', 'AR$': '9999.00', 'U$s': '27.36'}, {'id': 4, 'titulo': 'Born Y Quieto', 'url': 'https://www.cuspide.com//Libro/9789504979456/Born+Y+Quieto', 'AR$': '8900.00', 'U$s': '24.35'}, {'id': 5, 'titulo': 'Antes De Que Se Enfrie El Cafe', 'url': 'https://www.cuspide.com//Libro/9789506446567/Antes+De+Que+Se+Enfrie+El+Cafe', 'AR$': '7999.00', 'U$s': '21.89'}, {'id': 6, 'titulo': 'Habitos Atomicos', 'url': 'https://www.cu

Creo archivo csv de los libros sin precios (errores)

In [6]:
import pandas as pd

# Crear un DataFrame a partir de la lista de diccionarios
dfe = pd.DataFrame(errores)

# Nombre del archivo CSV
archivo_errores = "erroreslibros.csv"

# Guardar el DataFrame en un archivo CSV
dfe.to_csv(archivo_errores, index=False, sep=';')

Arreglo los id de los libros con precios para que queden en orden

In [7]:
idNuevo_libros = 0
indice = 0

for i in libros_precios:
    libros_precios[indice]['id'] = idNuevo_libros
print(libros_precios)

[{'id': 0, 'titulo': 'Este Dolor No Es Mio', 'url': 'https://www.cuspide.com//Libro/9788484456810/Este+Dolor+No+Es+Mio', 'AR$': '9900.00', 'U$s': '27.09'}, {'id': 1, 'titulo': 'Una Familia Anormal', 'url': 'https://www.cuspide.com//Libro/9789877364262/Una+Familia+Anormal', 'AR$': '4199.00', 'U$s': '11.49'}, {'id': 2, 'titulo': 'Adios Cachorra', 'url': 'https://www.cuspide.com//Libro/9789504979395/Adios+Cachorra', 'AR$': '6900.00', 'U$s': '18.88'}, {'id': 3, 'titulo': 'El Viento Conoce Mi Nombre', 'url': 'https://www.cuspide.com//Libro/9789500769013/El+Viento+Conoce+Mi+Nombre', 'AR$': '9999.00', 'U$s': '27.36'}, {'id': 4, 'titulo': 'Born Y Quieto', 'url': 'https://www.cuspide.com//Libro/9789504979456/Born+Y+Quieto', 'AR$': '8900.00', 'U$s': '24.35'}, {'id': 5, 'titulo': 'Antes De Que Se Enfrie El Cafe', 'url': 'https://www.cuspide.com//Libro/9789506446567/Antes+De+Que+Se+Enfrie+El+Cafe', 'AR$': '7999.00', 'U$s': '21.89'}, {'id': 6, 'titulo': 'Habitos Atomicos', 'url': 'https://www.cuspi

Web scraping del valor del dolar blue, con un ciclo en el que calcule el precio al dolar blue de cada libro y  la fecha actual, cada vez que se ejecute y acutalice va a decir la fecha en la que esto se hizo

In [9]:
import requests
from bs4 import BeautifulSoup
import re # Esta libreria me ayuda a eliminar los strings que retorne la busqueda, devolviendome solo el valor numerico del dolar
import datetime # Esta libreria es para trabajar con fechas y horas

urld = 'https://dolarhoy.com/'
responsed = requests.get(urld)
responsed.status_code == 200
soupd = BeautifulSoup(responsed.text, 'html.parser')
dolar_signo = soupd.find('div', class_='venta').get_text(strip=True).replace('.', '').replace(',', '.') # Esto retorna el precio pero con la palabra Venta y el signo $(peso)
dolar = float(re.search(r'\d+', dolar_signo).group()) # Variable que limpia el valor para que retorne solo los numeros y lo convierte en flotante
print('Valor dolar blue:',dolar)

precios_blue = []
for librod in libros_precios:
    preciopesos = librod['AR$']
    fecha_actual = datetime.date.today()

    precioblue = {
        'U$s_blue' : round((float(preciopesos) / dolar),2),
        'Fecha' : fecha_actual.strftime("%Y-%m-%d")
    }
    precios_blue.append(precioblue)
print(precios_blue)

Valor dolar blue: 738.0
[{'U$s_blue': 13.41, 'Fecha': '2023-08-29'}, {'U$s_blue': 5.69, 'Fecha': '2023-08-29'}, {'U$s_blue': 9.35, 'Fecha': '2023-08-29'}, {'U$s_blue': 13.55, 'Fecha': '2023-08-29'}, {'U$s_blue': 12.06, 'Fecha': '2023-08-29'}, {'U$s_blue': 10.84, 'Fecha': '2023-08-29'}, {'U$s_blue': 14.77, 'Fecha': '2023-08-29'}, {'U$s_blue': 6.64, 'Fecha': '2023-08-29'}, {'U$s_blue': 12.06, 'Fecha': '2023-08-29'}, {'U$s_blue': 12.06, 'Fecha': '2023-08-29'}, {'U$s_blue': 10.09, 'Fecha': '2023-08-29'}, {'U$s_blue': 12.19, 'Fecha': '2023-08-29'}, {'U$s_blue': 10.7, 'Fecha': '2023-08-29'}, {'U$s_blue': 11.52, 'Fecha': '2023-08-29'}, {'U$s_blue': 11.52, 'Fecha': '2023-08-29'}, {'U$s_blue': 8.81, 'Fecha': '2023-08-29'}, {'U$s_blue': 12.06, 'Fecha': '2023-08-29'}, {'U$s_blue': 11.79, 'Fecha': '2023-08-29'}, {'U$s_blue': 7.45, 'Fecha': '2023-08-29'}, {'U$s_blue': 11.52, 'Fecha': '2023-08-29'}, {'U$s_blue': 8.81, 'Fecha': '2023-08-29'}, {'U$s_blue': 11.52, 'Fecha': '2023-08-29'}, {'U$s_blue': 8

Combino los diccionarios de los titulos y precios (AR$ y U$s) con el diccionario que contiene los precios en dolar blue

In [10]:
# Lista con todos los datos necesarios de cada libro

libros_completo = []

for dicclibros_precios, diccprecios_blue in zip(libros_precios, precios_blue):
    dicclibros_precios.update(diccprecios_blue)
    libros_completo.append(dicclibros_precios)

print(libros_completo)

[{'id': 0, 'titulo': 'Este Dolor No Es Mio', 'url': 'https://www.cuspide.com//Libro/9788484456810/Este+Dolor+No+Es+Mio', 'AR$': '9900.00', 'U$s': '27.09', 'U$s_blue': 13.41, 'Fecha': '2023-08-29'}, {'id': 1, 'titulo': 'Una Familia Anormal', 'url': 'https://www.cuspide.com//Libro/9789877364262/Una+Familia+Anormal', 'AR$': '4199.00', 'U$s': '11.49', 'U$s_blue': 5.69, 'Fecha': '2023-08-29'}, {'id': 2, 'titulo': 'Adios Cachorra', 'url': 'https://www.cuspide.com//Libro/9789504979395/Adios+Cachorra', 'AR$': '6900.00', 'U$s': '18.88', 'U$s_blue': 9.35, 'Fecha': '2023-08-29'}, {'id': 3, 'titulo': 'El Viento Conoce Mi Nombre', 'url': 'https://www.cuspide.com//Libro/9789500769013/El+Viento+Conoce+Mi+Nombre', 'AR$': '9999.00', 'U$s': '27.36', 'U$s_blue': 13.55, 'Fecha': '2023-08-29'}, {'id': 4, 'titulo': 'Born Y Quieto', 'url': 'https://www.cuspide.com//Libro/9789504979456/Born+Y+Quieto', 'AR$': '8900.00', 'U$s': '24.35', 'U$s_blue': 12.06, 'Fecha': '2023-08-29'}, {'id': 5, 'titulo': 'Antes De Qu

Convierto la lista libros_completos en un dataframe, para posteriormente pasarlo a archivos csv

In [11]:
import pandas as pd

# Crear un DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(libros_completo)

# Nombre del archivo CSV
nombre_archivo = "webscrapinglibros.csv"

# Guardar el DataFrame en un archivo CSV
df.to_csv(nombre_archivo, index=False, sep=';')